In [6]:
import json
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


In [1]:
import pandas as pd
import numpy as np
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
import string
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

with open('intents.json', 'r') as file:
    data = json.load(file)

questions = []
answers =[]
# Перебираем все интенты в списке
for intent in data['intents']:
    # Получаем список паттернов для данного интента и добавляем его к общему массиву
    questions.extend(intent['patterns'])
    answers.extend(intent['responses'])

test_question = "What is a database?"

questions.append(test_question)
fitered_questions = []

for text in questions:
    doc = nlp(text)
    fitered_questions.append([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

fitered_questions = [' '.join(text) for text in fitered_questions]

cv = CountVectorizer()
questions_cv = cv.fit_transform(fitered_questions)

questions_cv = pd.DataFrame(questions_cv.toarray(), columns=cv.get_feature_names_out())

tfidf = TfidfVectorizer()
questions_tfidf = tfidf.fit_transform(fitered_questions)
questions_tfidf = pd.DataFrame(questions_tfidf.toarray(), columns=cv.get_feature_names_out())

questions_cv = questions_cv.to_numpy()
questions_tfidf = questions_tfidf.to_numpy()

In [7]:
with open('intents.json', 'r') as file:
    data = json.load(file)

# Подготовим данные
lemmatizer = WordNetLemmatizer()
# nltk.download('punkt')

In [8]:
def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

patterns = []
tags = []

for intent in data['intents']:
    patterns.extend(intent['patterns'])
    tags.extend([intent['tag']] * len(intent['patterns']))

processed_patterns = [preprocess_text(pattern) for pattern in patterns]

In [9]:
count_vectorizer = CountVectorizer()
bow_matrix = count_vectorizer.fit_transform(processed_patterns)
bow_matrix = pd.DataFrame(bow_matrix.toarray(), columns=count_vectorizer.get_feature_names_out())
bow_matrix = bow_matrix.to_numpy()


In [10]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_patterns)
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_matrix = tfidf_matrix.to_numpy()

In [13]:
# Функция для поиска наиболее схожего вопроса
def find_most_similar_question(user_input, matrix, vectorizer, similarity_metric):
    processed_input = preprocess_text(user_input)
    input_vector = vectorizer.transform([processed_input])

    # Вычислим схожесть с использованием выбранной метрики
    similarities = similarity_metric(matrix, input_vector)

    # Найдем индекс наиболее схожего вопроса
    most_similar_index = similarities.argmax()

    return patterns[most_similar_index], tags[most_similar_index]

# Пример использования с косинусным расстоянием
user_input = "What mean db?"
most_similar_pattern, tag = find_most_similar_question(user_input, bow_matrix, count_vectorizer, cosine_similarity)

# Выведем ответ
for intent in data['intents']:
    if intent['tag'] == tag:
        print(intent['responses'][0])
        break


A database management system (DBMS) is a software application that is used to create, manage, and manipulate databases. A database is a collection of data that is organized in a specific way, allowing for efficient retrieval and manipulation of the data. A DBMS provides a set of tools and interfaces that allow users to create, modify, and query the database, as well as to control access to the data and maintain the integrity and consistency of the data. DBMSs are widely used in a variety of applications, including financial systems, customer relationship management systems, and online shopping systems. They are an essential component of many business and organization systems, as they allow for the efficient storage and management of large amounts of data.


In [14]:

most_similar_pattern, tag = find_most_similar_question(user_input, tfidf_matrix, tfidf_vectorizer, cosine_similarity)

# Выведем ответ
for intent in data['intents']:
    if intent['tag'] == tag:
        print(intent['responses'][0])
        break

Programming is the process of creating computer software by writing instructions that can be executed by a computer. Different programming languages include Python, Java, C++, and JavaScript. Best practices in programming include writing clean and readable code, using version control, following coding standards, and testing and debugging thoroughly.


In [2]:
from sklearn.metrics.cluster import rand_score
#Поиск ближайшего ответа с помощью коэффициента сходства Рэнда (Rand Index)
# Коэффициент сходства Рэнда измеряет сходство между двумя кластеризациями или разбиениями. 
# В контексте поиска ближайшего ответа, мы используем его для измерения сходства между двумя наборами ответов.
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

best_sim = []
for i in range(len(questions_cv)-1):
    best_sim.append(rand_score(questions_cv[i,:], questions_cv[-1,:]))
print(answers[best_sim.index(max(best_sim))])

best_sim = []
for i in range(len(questions_tfidf)-1):
    best_sim.append(rand_score(questions_tfidf[i,:], questions_tfidf[-1,:]))
print(answers[best_sim.index(max(best_sim))])

Data abstraction is a technique used in programming to hide the implementation details of data types and only expose their essential properties and behaviors. It allows programmers to create abstract data types (ADTs) that define the interface and operations of a data type without revealing how it is implemented. This separation of interface from implementation allows for better code maintainability, flexibility, and modularity.
Data abstraction is a technique used in programming to hide the implementation details of data types and only expose their essential properties and behaviors. It allows programmers to create abstract data types (ADTs) that define the interface and operations of a data type without revealing how it is implemented. This separation of interface from implementation allows for better code maintainability, flexibility, and modularity.


In [3]:
from scipy.spatial import distance
# Поиск ближайшего ответа с помощью Жаккара (Jaccard Similarity)
# Жаккардово сходство измеряет схожесть между двумя множествами и 
# выражается отношением размера их пересечения к размеру их объединения.

best_sim = []
for i in range(len(questions_cv)-1):
    best_sim.append(1 - distance.jaccard(questions_cv[i, :], questions_cv[-1, :]))
print(answers[best_sim.index(max(best_sim))])

best_sim = []
for i in range(len(questions_tfidf)-1):
    best_sim.append(1 - distance.jaccard(questions_tfidf[i, :], questions_tfidf[-1, :]))
print(answers[best_sim.index(max(best_sim))])


Data abstraction is a technique used in programming to hide the implementation details of data types and only expose their essential properties and behaviors. It allows programmers to create abstract data types (ADTs) that define the interface and operations of a data type without revealing how it is implemented. This separation of interface from implementation allows for better code maintainability, flexibility, and modularity.
Data abstraction is a technique used in programming to hide the implementation details of data types and only expose their essential properties and behaviors. It allows programmers to create abstract data types (ADTs) that define the interface and operations of a data type without revealing how it is implemented. This separation of interface from implementation allows for better code maintainability, flexibility, and modularity.


In [4]:
from scipy.stats import pearsonr
#Поиск ближайшего ответа с помощью корреляции
# Корреляция измеряет степень линейной зависимости между двумя переменными. 
#  В данном случае, между векторами ответов и векторами запросов.
best_sim = []
for i in range(len(questions_cv)-1):
    best_sim.append(pearsonr(questions_cv[i,:], questions_cv[-1,:]))
print(answers[best_sim.index(max(best_sim))])

best_sim = []
for i in range(len(questions_tfidf)-1):
    best_sim.append(pearsonr(questions_tfidf[i,:], questions_tfidf[-1,:]))
print(answers[best_sim.index(max(best_sim))])

Data abstraction is a technique used in programming to hide the implementation details of data types and only expose their essential properties and behaviors. It allows programmers to create abstract data types (ADTs) that define the interface and operations of a data type without revealing how it is implemented. This separation of interface from implementation allows for better code maintainability, flexibility, and modularity.
Data abstraction is a technique used in programming to hide the implementation details of data types and only expose their essential properties and behaviors. It allows programmers to create abstract data types (ADTs) that define the interface and operations of a data type without revealing how it is implemented. This separation of interface from implementation allows for better code maintainability, flexibility, and modularity.
